<a href="https://colab.research.google.com/github/DavidTorres-sys/proyecto-modelos-y-simulacion-udea/blob/main/preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

In [49]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

In [50]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.28GB/s]


In [51]:
!unzip udea*.zip > /dev/null
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  59185238 test.csv
   692501  10666231 143732437 train.csv
  1286075  15528571 207634348 total


In [52]:
def read_file(file='train.csv'):
    try:
        return pd.read_csv(file)
    except FileNotFoundError:
        print(f"Error: El archivo '{file}' no se encontró.")
    except Exception as e:
        print(f"Ocurrió un error al leer el archivo: {e}")


In [53]:
df = read_file()
print("Tamaño del dataset:", df.shape)
df.head()

Tamaño del dataset: (692500, 21)


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


In [54]:
# ============================================================
# 3. Inspeccionar y limpiar datos
# ============================================================

# Mostrar valores faltantes
print("Valores faltantes por columna:")
print(df.isna().sum())

# Eliminar columnas completamente vacías
df = df.dropna(axis=1, how='all')

# Limpiar nombres de columnas
df.columns = df.columns.str.strip().str.replace('.', '_').str.upper()

print("Forma después de limpieza inicial:", df.shape)


Valores faltantes por columna:
ID                                 0
PERIODO_ACADEMICO                  0
E_PRGM_ACADEMICO                   0
E_PRGM_DEPARTAMENTO                0
E_VALORMATRICULAUNIVERSIDAD     6287
E_HORASSEMANATRABAJA           30857
F_ESTRATOVIVIENDA              32137
F_TIENEINTERNET                26629
F_EDUCACIONPADRE               23178
F_TIENELAVADORA                39773
F_TIENEAUTOMOVIL               43623
E_PRIVADO_LIBERTAD                 0
E_PAGOMATRICULAPROPIO           6498
F_TIENECOMPUTADOR              38103
F_TIENEINTERNET.1              26629
F_EDUCACIONMADRE               23664
RENDIMIENTO_GLOBAL                 0
INDICADOR_1                        0
INDICADOR_2                        0
INDICADOR_3                        0
INDICADOR_4                        0
dtype: int64
Forma después de limpieza inicial: (692500, 21)


In [55]:
# ============================================================
# 4. Conversión de rangos textuales y ordinales (versión corregida)
# ============================================================

# --- Rango de matrícula ---
valor_map = {
    "Menos de 500 mil": 0.25,
    "Entre 500 mil y menos de 1 millón": 0.75,
    "Entre 1 millón y menos de 2.5 millones": 1.75,
    "Entre 2.5 millones y menos de 4 millones": 3.25,
    "Entre 4 millones y menos de 5.5 millones": 4.75,
    "Entre 5.5 millones y menos de 7 millones": 6.25,
    "7 millones o más": 7.5
}

df["E_VALORMATRICULAUNIVERSIDAD"] = df["E_VALORMATRICULAUNIVERSIDAD"].map(valor_map)
df["E_VALORMATRICULAUNIVERSIDAD"] = df["E_VALORMATRICULAUNIVERSIDAD"].fillna(df["E_VALORMATRICULAUNIVERSIDAD"].median())

# --- Rango de horas de trabajo ---
horas_map = {
    "0": 0,
    "Menos de 10 horas": 5,
    "Entre 11 y 20 horas": 15,
    "Entre 21 y 30 horas": 25,
    "Más de 30 horas": 35
}
df["E_HORASSEMANATRABAJA"] = df["E_HORASSEMANATRABAJA"].map(horas_map)

# --- Estrato de vivienda ---
df["F_ESTRATOVIVIENDA"] = df["F_ESTRATOVIVIENDA"].str.extract(r"(\d+)").astype(float)
df["F_ESTRATOVIVIENDA"] = df["F_ESTRATOVIVIENDA"].fillna(df["F_ESTRATOVIVIENDA"].median())


In [56]:
# ============================================================
# 5. Conversión de variables binarias (Sí/No) con manejo robusto
# ============================================================

binarias = [
    "F_TIENEINTERNET", "F_TIENELAVADORA", "F_TIENEAUTOMOVIL",
    "E_PRIVADO_LIBERTAD", "E_PAGOMATRICULAPROPIO",
    "F_TIENECOMPUTADOR",
]

for col in binarias:
    if col in df.columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.strip()
            .str.replace("Sí", "Si", regex=False)
            .map({"Si": 1, "No": 0, "S": 1, "N": 0})
        )
        # Rellenamos los NaN con 0 (sin usar inplace)
        df[col] = df[col].fillna(0)

print("✅ Variables binarias convertidas correctamente.")


✅ Variables binarias convertidas correctamente.


In [57]:
# ============================================================
# 6. Manejo de valores faltantes (numéricos y categóricos)
# ============================================================

# Separar numéricas y categóricas nuevamente
num_cols = df.select_dtypes(include=["int64", "float64"]).columns
cat_cols = df.select_dtypes(include=["object"]).columns

# Imputar numéricas con la mediana
# Usamos try/except para ignorar columnas sin valores válidos
imputer_num = SimpleImputer(strategy="median")
try:
    df[num_cols] = imputer_num.fit_transform(df[num_cols])
except ValueError:
    # Si algunas columnas no tienen valores válidos, las llenamos manualmente
    for col in num_cols:
        if df[col].isna().all():
            df[col] = 0
        else:
            df[col].fillna(df[col].median(), inplace=True)

# Imputar categóricas con el valor más frecuente
if len(cat_cols) > 0:
    imputer_cat = SimpleImputer(strategy="most_frequent")
    df[cat_cols] = imputer_cat.fit_transform(df[cat_cols])

print("✅ Imputación de valores faltantes completada.")

✅ Imputación de valores faltantes completada.


In [58]:
# ============================================================
# 7. One-Hot Encoding de variables categóricas
# ============================================================

df_encoded = pd.get_dummies(df, columns=cat_cols, drop_first=True)
print("Forma después del one-hot encoding:", df_encoded.shape)


Forma después del one-hot encoding: (692500, 1018)


In [59]:
# ============================================================
# 8. Escalamiento de variables numéricas
# ============================================================

scaler = StandardScaler()
df_encoded[num_cols] = scaler.fit_transform(df_encoded[num_cols])

print("✅ Escalamiento completado.")


✅ Escalamiento completado.


In [60]:
df_encoded

,ID,PERIODO_ACADEMICO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,...,F_EDUCACIONMADRE_Postgrado,F_EDUCACIONMADRE_Primaria completa,F_EDUCACIONMADRE_Primaria incompleta,F_EDUCACIONMADRE_Secundaria (Bachillerato) completa,F_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,F_EDUCACIONMADRE_Técnica o tecnológica completa,F_EDUCACIONMADRE_Técnica o tecnológica incompleta,RENDIMIENTO_GLOBAL_bajo,RENDIMIENTO_GLOBAL_medio-alto,RENDIMIENTO_GLOBAL_medio-bajo
0,1.434424,1.294094,2.490198,-1.11686,0.452949,0.410790,0.478713,1.389589,-0.007007,-0.884073,...,True,False,False,False,False,False,False,False,True,False
1,0.527513,1.294094,0.603334,-1.48420,0.452949,-2.434331,0.478713,-0.719637,-0.007007,-0.884073,...,False,False,False,False,False,False,True,True,False,False
2,-0.652132,0.439801,0.603334,1.08718,0.452949,0.410790,0.478713,-0.719637,-0.007007,-0.884073,...,False,False,False,True,False,False,False,True,False,False
3,-0.084924,-0.319570,1.546766,-1.48420,1.371357,0.410790,0.478713,-0.719637,-0.007007,-0.884073,...,False,False,False,True,False,False,False,False,False,False
4,1.731274,1.294094,0.603334,0.35250,0.452949,0.410790,0.478713,1.389589,-0.007007,-0.884073,...,False,True,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,-1.644029,-0.319570,-0.969053,-0.38218,-0.465458,0.410790,0.478713,-0.719637,-0.007007,1.131129,...,False,False,False,False,True,False,False,False,True,False
692496,0.909035,1.294094,0.603334,1.08718,0.452949,0.410790,0.478713,-0.719637,-0.007007,-0.884073,...,False,False,False,False,True,False,False,True,False,False
692497,0.033541,-1.458627,-0.340098,-1.11686,0.452949,0.410790,0.478713,-0.719637,-0.007007,1.131129,...,False,False,False,False,True,False,False,False,False,True
692498,1.722828,-0.319570,0.603334,-1.11686,-1.383866,-2.434331,-2.088934,-0.719637,-0.007007,1.131129,...,False,True,False,False,False,False,False,True,False,False


In [61]:
#!kaggle competitions submit -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia -f rendimiento_por_id.csv -m "raul ramos submission with linear model"